In [1]:
import torch
import re
def xprint(s):
    s = str(s)
    s = re.sub(' *#.*','',s)
    print(s)

In [2]:
def myfn(x):
    y = x[0]
    for i in range(1, x.size(0)):
        y = y + x[i]
    return y

In [3]:
#追蹤法
inp = torch.randn(5,5)
traced_fn = torch.jit.trace(myfn, inp)
print(traced_fn.code) #迴圈被固定為 4 次了

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  y0 = torch.add(y, torch.select(x, 0, 1))
  y1 = torch.add(y0, torch.select(x, 0, 2))
  y2 = torch.add(y1, torch.select(x, 0, 3))
  return torch.add(y2, torch.select(x, 0, 4))



In [4]:
#抄寫法
scripted_fn = torch.jit.script(myfn)
print(scripted_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  _0 = torch.__range_length(1, torch.size(x, 0), 1)
  y0 = y
  for _1 in range(_0):
    i = torch.__derive_index(_1, 1, 1)
    y1 = torch.add(y0, torch.select(x, 0, i))
    y0 = y1
  return y0



In [5]:
xprint(scripted_fn.graph)

graph(%x.1 : Tensor):
  %10 : bool = prim::Constant[value=1]()
  %2 : int = prim::Constant[value=0]()
  %5 : int = prim::Constant[value=1]()
  %y.1 : Tensor = aten::select(%x.1, %2, %2)
  %7 : int = aten::size(%x.1, %2)
  %9 : int = aten::__range_length(%5, %7, %5)
  %y : Tensor = prim::Loop(%9, %10, %y.1)
    block0(%11 : int, %y.11 : Tensor):
      %i.1 : int = aten::__derive_index(%11, %5, %5)
      %19 : Tensor = aten::select(%x.1, %2, %i.1)
      %y.5 : Tensor = aten::add(%y.11, %19, %5)
      -> (%10, %y.5)
  return (%y)

